In [1]:
from models.arcs import load_alt_model_a, load_inf_model, GenPhiloText
from utilities.loaders import load_file
from utilities.preprocessors import preprocess, map_value_to_index, init_sequences_a, init_sequences_b, decode_predictions
from utilities.visualizers import export_results

from tensorflow.keras.losses import CategoricalCrossentropy as cce_loss
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import CategoricalAccuracy, CategoricalCrossentropy as cce_metric

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import tensorflow as tf

%load_ext autoreload
%autoreload 2

In [2]:
corpus = load_file('./data/notes.txt')

In [3]:
corpus[:500]

'A simple idea “What is the meaning of life?”\nI asked as I learned through the works of Camus? \nOne step down, I felt a yearning of meaning in this world.\nIn this yearning I stumbled upon eastern philosophy;\nIkigai as the Japanese philosophers called it was a considerable way for me to find meaning at that certain point in my life. \nFollowed then another idea, a leap of faith as Kierkegaard would call it, yet I had no idea this was his idea. \nCalm followed after the storm, and then I took another'

In [4]:
len(corpus)

226750

In [5]:
chars = sorted(list(set(corpus)))
chars

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '´',
 'ç',
 'é',
 'ï',
 '–',
 '—',
 '‘',
 '’',
 '“',
 '”',
 '…']

# Preprocessing corpus
* replace quotation marks like this '“'/'”' with this instead '"'
* replace single quotation marks like this '‘'/'’' with ''' instead
* replace this hyphen '–' with this hyphen '—'
* lowercase all words (for now)
* replace 3 consecutive '.' with  '…' instead

In [6]:
corpus = preprocess(corpus)
corpus[:500]

'a simple idea "what is the meaning of life?"\ni asked as i learned through the works of camus? \none step down, i felt a yearning of meaning in this world.\nin this yearning i stumbled upon eastern philosophy;\nikigai as the japanese philosophers called it was a considerable way for me to find meaning at that certain point in my life. \nfollowed then another idea, a leap of faith as kierkegaard would call it, yet i had no idea this was his idea. \ncalm followed after the storm, and then i took another'

In [7]:
len(corpus)

226961

In [8]:
chars = sorted(list(set(corpus)))
# chars = ['[UNK]'] + chars
chars

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '´',
 'ç',
 'é',
 'ï',
 '—',
 '…']

# Creating mapper from a unique character to its respective index

In [9]:
char_to_idx = map_value_to_index(chars)
idx_to_char = map_value_to_index(chars, inverted=True)

In [10]:
char_to_idx

In [11]:
idx_to_char

In [12]:
n_unique = len(char_to_idx.get_vocabulary())
n_unique

57

In [13]:
n_time_steps = 100
X, Y = init_sequences_a(corpus, char_to_idx, T_x=n_time_steps)
X

<tf.Tensor: shape=(226861, 100), dtype=int64, numpy=
array([[25,  2, 43, ..., 29,  2, 43],
       [ 2, 43, 33, ...,  2, 43, 44],
       [43, 33, 37, ..., 43, 44, 29],
       ...,
       [39, 42,  2, ..., 29, 36, 33],
       [42,  2, 44, ..., 36, 33, 29],
       [ 2, 44, 42, ..., 33, 29, 30]], dtype=int64)>

In [14]:
X.shape

TensorShape([226861, 100])

In [15]:
Y

<tf.Tensor: shape=(226861,), dtype=int64, numpy=array([44, 29, 40, ..., 29, 30, 11], dtype=int64)>

# convert Y data's indeces to their one hot vector representation

In [16]:
Y = tf.one_hot(Y, depth=n_unique)
Y

<tf.Tensor: shape=(226861, 57), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [17]:
Y[3]

<tf.Tensor: shape=(57,), dtype=float32, numpy=
array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.], dtype=float32)>

In [18]:
# number of examples
len(X)

226861

In [19]:
len(X[-1])

100

# Instantiate generative model A with set architecture

In [20]:
model = load_alt_model_a(n_unique=n_unique, T_x=n_time_steps, emb_dim=32, n_a=128)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 32)           1824      
                                                                 
 lstm (LSTM)                 (None, 100, 128)          82432     
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 57)                7353      
                                                                 
 batch_normalization (Batch  (None, 57)                228       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 57)                0         
                                                        

# Provide loss, optimizer, and metrics for both alternative models A and B

In [21]:
opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
loss = cce_loss()
metrics = [CategoricalAccuracy(), cce_metric()]

# model.compile(loss=loss, optimizer=opt, metrics=metrics)

# Train alternative model A and checkpoint weights

In [22]:
# weights_path = "./weights/weights-improvement-{epoch:02d}-{categorical_accuracy:.4f}.hdf5"
# checkpoint = ModelCheckpoint(weights_path, monitor='categorical_accuracy', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]

In [23]:
# history = model.fit(X, Y, epochs=20, batch_size=2048, callbacks=callbacks_list)

In [24]:
# export_results(history, ['loss'], image_only=False)
# export_results(history, ['categorical_accuracy'], image_only=False)

# Preprocessing for alternative model B

In [25]:
X, Y = init_sequences_b(corpus, char_to_idx, T_x=n_time_steps)
X

<tf.Tensor: shape=(2248, 100), dtype=int64, numpy=
array([[25,  2, 43, ..., 29,  2, 43],
       [29, 40,  2, ..., 36, 39, 43],
       [40, 32, 49, ..., 25, 44,  2],
       ...,
       [39,  2, 30, ..., 32, 33, 43],
       [44, 32, 29, ...,  2, 26, 42],
       [25, 35, 43, ...,  0,  0,  0]], dtype=int64)>

In [26]:
X.shape[0]

2248

In [27]:
len(X)

2248

In [28]:
len(X[-2])

100

In [29]:
len(X[-1])

100

In [30]:
len(Y[-1])

100

In [31]:
X[-1]

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([25, 35, 43,  2, 37, 49,  2, 26, 29, 36, 33, 29, 30,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int64)>

In [32]:
Y

<tf.Tensor: shape=(2248, 100), dtype=int64, numpy=
array([[ 2, 43, 33, ...,  2, 43, 44],
       [40,  2, 28, ..., 39, 43, 39],
       [32, 49, 23, ..., 44,  2, 44],
       ...,
       [ 2, 30, 33, ..., 33, 43, 56],
       [32, 29,  2, ..., 26, 42, 29],
       [35, 43,  2, ...,  0,  0,  0]], dtype=int64)>

In [33]:
Y = [tf.one_hot(y, depth=n_unique) for y in tf.reshape(Y, shape=(-1, Y.shape[0]))]

In [34]:
len(Y)

100

# Y here is now a matrix with shape $(T_y, m, n_{unique})$

In [35]:
Y

[<tf.Tensor: shape=(2248, 57), dtype=float32, numpy=
 array([[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(2248, 57), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(2248, 57), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(2248, 57), dtype=float32, 

# Load a saved model
* see arhictecure
* see if prediction will work properly on dummy data

In [36]:
# saved_model = load_model('./saved/models/test_model_b.h5')

In [37]:
# saved_model.summary()

#### Recall that our model needs 3 inputs, X, the hidden state, and the cell state. Because we are generating novel sequences using our trained model we pass in a $(1, 100)$ input where it represents the shape $(m, T_x)$, moreover our hidden and cell states remain the same in terms of their shape which is $(m, n_a)$ but only now it would be $(1, n_a)$ since we are passing only one input example to our model

In [38]:
sample_input = tf.random.uniform(shape=(1, 100), minval=0, maxval=n_unique - 1, dtype=tf.int32)

sample_h = tf.zeros(shape=(1, 64))
sample_c = tf.zeros(shape=(1, 64))

#### use the model to predict an output Y which we know will be of shape $(T_y, m, n_{unique})$ or in this case since we only inputted one example $(T_y, 1, 26)$

In [39]:
# saved_model.predict([sample_input, sample_h, sample_c])

#### List all layer names
* the goal here is to extract the Embedding, LSTM, Dense, and BatchNormalization layers used in training the model which have all been instantiated once and where we can extract and use as objects in our inference model
* Once we know the layers, we can start by training our model for training, and save the best model with the lowest loss value
* Access the saved models aforementioned layers and use it in the inference model

In [40]:
# layers = saved_model.layers
# for layer in layers:
#     print(layer.name)

In [41]:
# lstm_cell = saved_model.get_layer('lstm-cell')
# embedding_layer = saved_model.get_layer('character-lookup')
# dense_layers = []
# norm_layers = []
# for layer in layers:
#     if "dense" in layer.name:
#         dense_layers.append(saved_model.get_layer(layer.name))
#     if "norm" in layer.name:
#         norm_layers.append(saved_model.get_layer(layer.name))

In [42]:
# lstm_cell.get_weights()

In [43]:
# embedding_layer.get_weights()

In [44]:
# for dense_layer in dense_layers:
#     print(dense_layer.get_weights())

In [45]:
# for norm_layer in norm_layers:
#     print(norm_layer.get_weights())

# Load saved weights

In [46]:
saved_model = GenPhiloText(emb_dim=32, n_a=64, n_unique=57, T_x=100, dense_layers_dims=[64, 32, 57])
saved_model([sample_input, sample_h, sample_c])

[<tf.Tensor: shape=(1, 57), dtype=float32, numpy=
 array([[ 4.3856274e-04, -2.3498174e-03, -2.8684244e-05,  6.5974054e-06,
         -3.0625749e-03, -5.8499450e-04,  1.5860591e-03, -4.4780463e-04,
         -2.7863425e-04, -9.6861529e-04, -1.4875045e-03,  1.7751012e-03,
          2.7695824e-03,  1.7940128e-03, -1.8161664e-03, -1.0306839e-03,
         -1.5878143e-03,  2.1795859e-03, -5.1378465e-04, -2.6313862e-04,
          2.0419441e-03,  2.3972145e-03,  3.0250733e-03, -2.5952701e-03,
          8.9494482e-05, -2.0019431e-03,  1.0309914e-03,  1.6091286e-03,
         -1.9030691e-03,  1.7768949e-04, -2.4148843e-03,  9.5656596e-04,
          4.9124419e-04, -3.7375921e-03,  2.1968529e-04, -3.5582317e-04,
         -9.3946012e-04, -1.7655373e-03, -5.8104267e-04, -4.9121433e-04,
         -9.2755072e-05, -1.3128530e-03,  5.9790909e-04, -1.3559521e-03,
          4.0295668e-04,  1.5279746e-03, -1.5458965e-03,  1.0647223e-04,
          7.9822436e-04,  2.1094196e-03, -4.3721983e-04,  5.9420872e-04,
 

In [47]:
saved_model.summary()

Model: "gen_philo_text"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 char-emb-layer (Embedding)  multiple                  1824      
                                                                 
 lstm-cell (LSTM)            multiple                  24832     
                                                                 
 dense-layer-0 (Dense)       multiple                  4160      
                                                                 
 dense-layer-1 (Dense)       multiple                  2080      
                                                                 
 dense-layer-2 (Dense)       multiple                  1881      
                                                                 
 norm-layer-0 (BatchNormali  multiple                  256       
 zation)                                                         
                                                    

In [49]:
saved_model.load_weights(filepath='./saved/weights/test_gen_philo_text_34_404.3077.h5')

In [50]:
saved_model.predict([sample_input, sample_h, sample_c])

1/1 [==============================] - 50s 50s/step


[array([[ 0.00198855, -0.00558211,  0.00968151,  0.00759611, -0.00687135,
          0.00137851, -0.00678063, -0.00881831,  0.01678021,  0.00372551,
          0.00400432, -0.00372502,  0.01171082,  0.00085732,  0.00827243,
          0.00225648,  0.00755347,  0.01042957,  0.00713646,  0.00690729,
         -0.00218364, -0.00637149,  0.00179633,  0.00503279,  0.00555751,
          0.0137592 ,  0.00722215,  0.00055222,  0.00121811,  0.00930299,
          0.00478258,  0.00626105, -0.00424836,  0.00904589,  0.00611137,
          0.00391184, -0.01038935, -0.00780329, -0.00451375, -0.01019667,
          0.00863175,  0.01598281, -0.00083186, -0.00364014, -0.00379593,
          0.00259919,  0.00842131, -0.00705327, -0.00314405, -0.00253984,
         -0.00264687,  0.01570259,  0.00348838, -0.00749015, -0.00452111,
          0.00483069,  0.00853883]], dtype=float32),
 array([[ 0.00198855, -0.00558211,  0.00968151,  0.00759611, -0.00687135,
          0.00137851, -0.00678063, -0.00881831,  0.01678021

In [51]:
layers = saved_model.layers
for layer in layers:
    print(layer.name)

char-emb-layer
lstm-cell
dense-layer-0
dense-layer-1
dense-layer-2
norm-layer-0
norm-layer-1
act-layer-0
act-layer-1
drop-layer-0
drop-layer-1
reshape-layer


In [52]:
char_emb_layer = saved_model.get_layer('char-emb-layer')
char_emb_layer.get_weights()

[array([[-5.40271072e-09,  6.06722394e-16,  3.59989409e-14, ...,
          1.55720033e-12,  3.99154165e-16, -3.12085806e-13],
        [-4.55186453e-16,  1.10625395e-16, -1.65313150e-14, ...,
          8.55269157e-14,  1.50356363e-15,  1.01421885e-10],
        [ 1.81588581e-11,  1.27070756e-14,  3.60846471e-14, ...,
         -1.22042402e-11, -9.78438356e-16,  9.24125558e-08],
        ...,
        [-3.14320758e-17,  7.33369325e-14, -8.75533885e-17, ...,
          1.13722141e-16,  3.50820824e-15, -1.12827614e-13],
        [ 7.20674768e-13, -3.47102852e-17, -2.01032615e-12, ...,
          9.36540734e-10,  5.49492049e-11, -8.67940003e-07],
        [ 1.13455521e-11, -8.41436740e-18, -1.55178079e-14, ...,
          8.51403273e-18, -4.24577040e-12,  2.30075372e-08]], dtype=float32)]

In [53]:
lstm_cell = saved_model.get_layer('lstm-cell')
lstm_cell.get_weights()

[array([[-0.12579572,  0.01834889,  0.09828047, ...,  0.14519428,
         -0.10652614, -0.05688067],
        [-0.00527227, -0.03416812, -0.02956024, ..., -0.05795737,
         -0.00856181,  0.03253853],
        [-0.0680214 ,  0.02220491, -0.03431651, ...,  0.03389829,
         -0.07210495,  0.01295092],
        ...,
        [-0.02331153, -0.08689586,  0.09996857, ...,  0.01261597,
         -0.01175508,  0.03922412],
        [-0.09820881, -0.12220346,  0.12566575, ..., -0.01489718,
         -0.10304449,  0.10140569],
        [ 0.02749164, -0.07388938, -0.11912709, ..., -0.0497042 ,
         -0.08691522,  0.01166691]], dtype=float32),
 array([[-0.0083413 , -0.0275647 , -0.00052363, ...,  0.01948158,
          0.06690418, -0.0244772 ],
        [-0.1348597 , -0.13983186, -0.02580041, ..., -0.08989543,
         -0.08801004, -0.03146078],
        [ 0.02184679,  0.00167274, -0.00603506, ...,  0.07316469,
         -0.04019942,  0.09860177],
        ...,
        [ 0.11840196,  0.03873251,  0.0

In [54]:
dense_layers = []
norm_layers = []
for layer in layers:
    if "dense" in layer.name:
        dense_layers.append(saved_model.get_layer(layer.name))
    if "norm" in layer.name:
        norm_layers.append(saved_model.get_layer(layer.name))

In [55]:
for dense_layer in dense_layers:
    print(dense_layer.get_weights())

[array([[ 1.01044824e-12,  6.92518222e-12, -2.06278639e-23, ...,
        -2.35209824e-10, -1.69050285e-08, -4.01603054e-15],
       [-1.37678768e-07,  2.19264406e-22,  2.61913687e-19, ...,
         3.86301366e-16,  3.38137035e-15, -5.52358008e-23],
       [ 1.02285179e-15,  1.80927729e-20, -2.53610735e-23, ...,
         3.77321353e-23,  6.60068886e-23, -9.51746237e-10],
       ...,
       [-1.78986576e-23, -4.38597388e-11, -1.57685154e-23, ...,
         1.31567894e-23, -3.54480591e-07, -1.08691078e-23],
       [ 5.26062638e-10,  9.52256727e-08,  4.03096445e-11, ...,
         2.00815300e-20,  2.18421791e-11,  5.26660064e-18],
       [-3.37843020e-18, -2.14911207e-22,  3.27923988e-23, ...,
        -3.29913491e-19, -9.74514586e-24,  1.66568812e-18]], dtype=float32), array([-9.59965121e-03, -3.13743041e-03,  5.74947153e-05,  4.32170322e-03,
        9.74050537e-03,  6.96101086e-03,  8.13718792e-03, -4.63995663e-03,
        7.50286784e-03, -3.63560417e-03,  9.05132852e-03,  6.97822031e-03,
 

In [56]:
for norm_layer in norm_layers:
    print(norm_layer.get_weights())

[array([0.9902269 , 1.0036864 , 0.9586819 , 0.96943253, 0.96426874,
       0.9619019 , 0.960164  , 0.9619977 , 0.9505472 , 0.9546746 ,
       0.95681626, 0.95181483, 0.968196  , 0.95200247, 0.9546518 ,
       0.95596635, 0.94868267, 0.95994306, 0.9550517 , 0.9666462 ,
       0.9545736 , 0.98577565, 0.96508974, 0.9597461 , 0.95686525,
       0.96605456, 0.9496918 , 0.96644086, 0.98552996, 0.9719695 ,
       0.957549  , 0.96183556, 0.95694774, 0.9663198 , 0.96488154,
       0.960273  , 0.9571881 , 0.9835492 , 0.9595259 , 0.95766896,
       0.96951365, 1.0020788 , 0.97258216, 0.95891166, 0.9639335 ,
       0.9597528 , 0.95483214, 0.9655677 , 0.96248823, 0.9982244 ,
       0.9678425 , 0.98377055, 0.96156776, 0.9593908 , 0.94963175,
       0.9613592 , 0.9427803 , 0.96853924, 0.95389086, 0.9581381 ,
       0.9487026 , 0.9838977 , 0.95886785, 0.9925848 ], dtype=float32), array([-0.04681864, -0.02841648, -0.04611355, -0.04880665, -0.03887705,
       -0.04468152, -0.04809877, -0.04700251, -0.05

In [57]:
inference_model = load_inf_model(char_emb_layer, lstm_cell, dense_layers, norm_layers, char_to_idx)

57
64
tf.Tensor(
[[-inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]], shape=(1, 57), dtype=float32)
100
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [58]:
inference_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 char-emb-layer (Embedding)  (None, 1, 32)                1824      ['input_2[0][0]',             
                                                                     'reshape-layer[0][0]',       
                                                                     'reshape-layer[1][0]',       
                                                                     'reshape-layer[2][0]',       
                                                                     'reshape-layer[3][0]',       
                                                                     'reshape-layer[4][0]',   

In [65]:
sample_input = tf.random.uniform(shape=(1, 1), minval=0, maxval=n_unique - 1, dtype=tf.int32)

sample_h = tf.random.normal(shape=(1, 64))
sample_c = tf.random.normal(shape=(1, 64))

In [66]:
sample_input

<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[44]])>

In [67]:
pred_ids = inference_model.predict([sample_input, sample_h, sample_c])

1/1 [==============================] - 0s 53ms/step


In [69]:
pred_ids

[array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=int64),
 array([[8]], dtype=

In [70]:
decode_predictions(pred_ids, idx_to_char)

'))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))'